In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 
from tqdm import tqdm 
tqdm.pandas()

In [2]:
train_dset_df = pd.read_csv("final_preprocessed_train.csv")

In [3]:
train_dset_df.head()

,Unnamed: 0,qid,question_text,target,preprocessed
0,0,6f47b0f60633c2056455,"How can I reply to this comment, ""India is poo...",0,"['how', 'i', 'reply', 'comment', 'india', 'poo..."
1,1,d49b3966070b27bf07fc,What did they use for transportation in Ancien...,0,"['what', 'use', 'transportation', 'ancient', '..."
2,2,6d5faa49380557c8ca7b,What are the most important provisions of Obam...,0,"['what', 'important', 'provision', 'obamacare']"
3,3,cebea75faa47388edcf5,At what age do most Finns master English today?,0,"['at', 'age', 'finns', 'master', 'english', 't..."
4,4,2a7b76a679cadb0a016e,What is cheapest place to live in India for on...,0,"['what', 'cheap', 'place', 'live', 'india', 'm..."


In [4]:
train_dset_df.drop(axis="columns", labels="Unnamed: 0", inplace=True)

In [5]:
train_dset_df.preprocessed = train_dset_df.preprocessed.progress_apply(eval)

100%|██████████| 783673/783673 [00:10&lt;00:00, 78057.95it/s]


In [9]:
train_dset_df["preprocessed_joined"] = train_dset_df.preprocessed.progress_apply(lambda x: " ".join(x))

100%|██████████| 783673/783673 [00:00&lt;00:00, 1043981.70it/s]


In [10]:
grouped_train_dset_df = train_dset_df.groupby("target")
zeros_dset_df = grouped_train_dset_df.get_group(0)
ones_dset_df = grouped_train_dset_df.get_group(1)


In [11]:
list_of_unique_words_zero = sorted(list(set((" ".join(zeros_dset_df.preprocessed_joined.to_list())).split())))

In [12]:
list_of_unique_words_one = sorted(list(set((" ".join(ones_dset_df.preprocessed_joined.to_list())).split())))

In [13]:
with open("2020_10_06_list_of_unique_words_zero.txt", "w") as file:
    file.write(" ".join(list_of_unique_words_zero) + "\n")
with open("2020_10_06_list_of_unique_words_one.txt", "w") as file:
    file.write(" ".join(list_of_unique_words_one) + "\n")

In [14]:
string_of_words_zero = " ".join(zeros_dset_df.preprocessed_joined.to_list())
string_of_words_one = " ".join(ones_dset_df.preprocessed_joined.to_list())

In [15]:
import nltk

In [19]:
list_of_words_zero = string_of_words_zero.split()
list_of_words_one = string_of_words_one.split()

In [22]:
freq_dist_zero = nltk.probability.FreqDist(list_of_words_zero)

In [23]:
freq_dist_one = nltk.probability.FreqDist(list_of_words_one)

In [73]:
freq_dist_zero_df = pd.DataFrame(list(freq_dist_zero.items()), columns=["Word", "Frequency0"])
freq_dist_zero_df.set_index("Word")

,Frequency0
Word,
how,152928
i,191497
reply,511
comment,1006
india,17634
...,...
coleseed,1
oenone,1
anthurium,1


In [74]:
freq_dist_one_df = pd.DataFrame(list(freq_dist_one.items()), columns=["Word","Frequency1"])
freq_dist_one_df.set_index("Word")

,Frequency1
Word,
why,19814
girl,1919
become,969
escort,15
sex,1568
...,...
operations,1
suffocated,1
blondes,1


In [108]:
combined_frequency_data = pd.merge(freq_dist_zero_df, freq_dist_one_df, how="outer")

In [109]:
combined_frequency_data.set_index("Word", inplace=True)

In [110]:
combined_frequency_data.fillna(value=0, inplace=True)

In [111]:
combined_frequency_data.to_csv("2020_10_06_combined_word_frequency_data.csv")

In [112]:
combined_frequency_data["Total"] = combined_frequency_data.sum(axis="columns")

# Probabilistic analysis

## 1. In the corpus, what is the probability that a random word belongs to class (target == 1)?

In [113]:
combined_frequency_data

,Frequency0,Frequency1,Total
Word,,,
how,152928.0,4629.0,157557.0
i,191497.0,7819.0,199316.0
reply,511.0,39.0,550.0
comment,1006.0,160.0,1166.0
india,17634.0,1984.0,19618.0
...,...,...,...
thiruvalluvar,0.0,1.0,1.0
surging,0.0,1.0,1.0
prajatantra,0.0,1.0,1.0


In [115]:
sum_of_frequency_each_column = combined_frequency_data.sum(axis="rows")

In [116]:
probability_for_random_word = sum_of_frequency_each_column/sum_of_frequency_each_column["Total"]

In [117]:
probability_for_random_word

Frequency0    0.917654
Frequency1    0.082346
Total         1.000000
dtype: float64

## 2. Given a particular word, what is the probability that the word is class (target == 1)?